Main code

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
from datetime import datetime, timedelta
import re
from collections import deque

saved_message_ids = deque(maxlen=100)
saved_message_ids_set = set()

def parse_timestamp(timestamp_str, now):
    now = datetime.now()
    timestamp_str = timestamp_str.strip()
    if timestamp_str.startswith('Less than a minute ago'):
        # Format: Less than a minute ago
        timestamp = now
    elif re.match(r'^\d+m$', timestamp_str):
        # Format: Xm - x minutes ago
        minutes_ago = int(timestamp_str.rstrip('m'))
        timestamp = now - timedelta(minutes=minutes_ago)
    elif re.match(r'^\d+:\d+ [AP]M$', timestamp_str):
        # Format: XX:XX AM/PM
        timestamp = datetime.strptime(timestamp_str, '%I:%M %p')
        timestamp = timestamp.replace(year=now.year, month=now.month, day=now.day)
    elif timestamp_str.startswith('Yesterday'):
        # Format: Yesterday XX:XX AM/PM
        time_part = timestamp_str.replace('Yesterday', '').strip()
        timestamp = datetime.strptime(time_part, '%I:%M %p')
        yesterday = now - timedelta(days=1)
        timestamp = timestamp.replace(year=yesterday.year, month=yesterday.month, day=yesterday.day)
    else:
        # Format: Month DD, YYYY XX:XX AM/PM
        timestamp = datetime.strptime(timestamp_str, '%B %d, %Y %I:%M %p')
    
    return int(timestamp.timestamp())

# start_of_scraping and number of days take in only these formats:
# Xm - x minutes - for ex. 30m; this works only for the last one hour
# XX:XX AM/PM - for ex. 6:15 AM; this works only for today (the day of scraping)
# Yesterday XX:XX AM/PM - for ex. Yesterday 11:55 PM - for yesterday (the day before the day of scraping)
# Month DD, YYYY XX:XX AM/PM - for ex. May 25, 2024 6:15 AM - for any day in the past except today and yesterday

now = datetime.now()

start_of_scraping = None #parse_timestamp('5m', now)
number_of_days = parse_timestamp('2m', now)
check_last_x = -100

def add_message_id(message_id):
    if message_id not in saved_message_ids_set:
        if len(saved_message_ids) == saved_message_ids.maxlen:
            # Remove the oldest ID from both deque and set
            oldest_id = saved_message_ids.popleft()
            saved_message_ids_set.remove(oldest_id)
        saved_message_ids.append(message_id)
        saved_message_ids_set.add(message_id)
        return True  # New message
    return False  # Duplicate message

def save_messages_to_json(messages, filename): # Dump messages to the json file
    with open(filename, 'a', encoding='utf-8') as f:
        for message in messages:
            json.dump(message, f, ensure_ascii=False)
            f.write(',\n')

def log_into_account(driver, username, password):
    WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.NAME, 'login'))).send_keys(username)
    driver.find_element(By.NAME, 'password').send_keys(password)
    driver.find_element(By.XPATH, '//button[@data-testid="log-in-submit"]').click()
    WebDriverWait(driver, 10).until(EC.url_changes(driver.current_url))

def fast_scroll(driver):
    message = driver.find_elements(By.CSS_SELECTOR, 'div.StreamMessage_container__omTCg')[-1:]
    for i in message:
        timestamp = parse_timestamp(i.find_element(By.CSS_SELECTOR, 'time.StreamMessage_timestamp__VVDmF').text, now)
        if timestamp <= start_of_scraping:
            return True

    else:
        False

def scroll_and_scrape(driver, filename):
    messages = driver.find_elements(By.CSS_SELECTOR, 'div.StreamMessage_container__omTCg')[check_last_x:]
    new_messages = []
    found_desired_timestamp = False

    for message in messages:
        try:
            message_id = message.get_attribute('data-testid').replace('message-', '')
            if not add_message_id(message_id):
                continue  # Skip duplicate messages
            # Extract message details
            message_text = message.find_element(By.CSS_SELECTOR, 'div.RichTextMessage_body__4qUeP').text
            timestamp = parse_timestamp(message.find_element(By.CSS_SELECTOR, 'time.StreamMessage_timestamp__VVDmF').text, now)
            likes_elem = message.find_elements(By.CSS_SELECTOR, 'span.StreamMessageLabelCount_labelCount__dWyPL')
            likes = likes_elem[0].text if likes_elem[0].text != "" else '0'

            if timestamp <= number_of_days:
                found_desired_timestamp = True

            message_obj = {
                'id': message_id,
                'timestamp': timestamp,
                'likes': likes,
                'text': message_text,
            }
            new_messages.append(message_obj)

        except Exception:
            continue

    if new_messages:
        save_messages_to_json(new_messages, filename) # Save new messages

    return found_desired_timestamp

def remove_old_messages(driver): # Saves RAM and makes scraping much more efficient and faster
    # JavaScript to remove messages that are out of the viewport
    script = """
    var messages = document.querySelectorAll('div.StreamMessage_container__omTCg');
    var viewportHeight = window.innerHeight;
    messages.forEach(function(message) {
        var rect = message.getBoundingClientRect();
        if (rect.bottom < 0) {
            message.parentNode.removeChild(message);
        }
    });
    """
    driver.execute_script(script)
    driver.execute_script(script)
    elements = driver.find_elements(By.CSS_SELECTOR, '.LazyLoad.is-visible')
    for element in elements:
        driver.execute_script("""
            var elem = arguments[0];
            elem.parentNode.removeChild(elem);
        """, element)

def main():
    options = Options()
    options.headless = True
    driver = webdriver.Chrome(options=options)
    driver.get('https://stocktwits.com/signin?next=/login')

    with open('usrpass.txt', 'r') as file:
        username, password = [line.strip() for line in file.readlines()]

    log_into_account(driver, username, password)
    driver.get('https://stocktwits.com/stream/trending')

    json_filename = 'messages.json'

    try:
        with open(json_filename, 'r+', encoding='utf-8') as f:
            content = f.read().strip()
            if not content:
                f.write('[')
            else:
                f.seek(0)
                f.write(content.rstrip(']') + ',\n')
    except FileNotFoundError:
        with open(json_filename, 'w', encoding='utf-8') as f:
            f.write('[')

    fast_scroll_count = 0
    if start_of_scraping:
        print("Fast scrolling to the desired timestamp.")
        while not fast_scroll(driver):
            driver.execute_script("window.scrollBy(0, window.innerHeight);")
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.StreamMessage_container__omTCg'))
            )
            fast_scroll_count += 1
            if fast_scroll_count % 5 == 0:
                remove_old_messages(driver)
        remove_old_messages(driver)

    scroll_count = 0
    while True:
        if scroll_and_scrape(driver, json_filename):
            print("Reached desired timestamp.")
            break
        driver.execute_script("window.scrollBy(0, window.innerHeight);")
        scroll_count += 1

        # Remove old messages every few scrolls to reduce memory usage
        if scroll_count % 5 == 0:
            remove_old_messages(driver)

        # Wait for new messages to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, 'div.StreamMessage_container__omTCg'))
        )

    print("Scraping completed.")
    driver.quit()

    with open(json_filename, 'a', encoding='utf-8') as f:
        f.seek(0, 2)  # Move the cursor to the end of the file
        f.seek(f.tell() - 2, 0)  # Move the cursor back by 2 bytes to remove the last comma
        f.truncate()  # Truncate the file at the current cursor position
        f.write(']')

if __name__ == "__main__":
    main()